# Dependências

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### API KEY

In [35]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Dados

Aqui estaremos carregando um texto sobre `Agents` de um blog no GitHub para exemplificação. Este foi um exemplo tirado da própria documentação do LangChain.

In [8]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [9]:
len(docs)

1

In [11]:
docs[0]

Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final res

In [12]:
docs[0].page_content

'\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nS

In [16]:
len(docs[0].page_content)

43131

# Chunks e Embeddings
Nesse momento, entramos no foco desta etapa do tutorial que são: **chunks** e **embeddings**. Uma breve contextualização:

- `Chunks`: são os pedaços de texto provenientes da divisão do seu documento, basicamente.
- `Embeddings`: representações vetoriais númericas que incorporam semanticamente o conteúdo dos seus documentos (chunks).

Seguiremos o tutorial utilizando as ferramentas do LangChain.

### RecursiveCharacterTextSplitter

A biblioteca fornece uma gama de `text_splitters`, que são classes/ferramentas usadas para conseguirmos realizar a divisão dos nossos textos e gerar os tão famosos chunks. O mais básico e mais usual deles é o `RecursiveCharacterTextSplitter`, recomendado para texto genérico. É parametrizado por uma lista de caracteres. Vamos dar uma olhada em como nós instanciamos esta classe.

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

Vamos examinar os parâmetros definidos acima para o uso do **RecursiveCharacterTextSplitter**:

- `chunk_size`: O tamanho máximo do chunk, onde o tamanho é determinado pela length_function. Geralmente, utilizamos a própria função `len`, então será o número máximo de caracteres por chunk. 
- `chunk_overlap`: Sobreposição entre os chunks. A sobreposição de blocos ajuda a mitigar a perda de informações quando o contexto é dividido entre blocos. Controla o quanto cada chunk consecutivo terá em comum com o bloco anterior.
- `length_function`: Função que determina o tamanho do chunk.
- `is_separator_regex`: Se a lista de separadores (padrão ["\n\n", "\n", " ", ""]) deve ser interpretada como regex.

Agora que um ``text_splitter`` foi instanciado, podemos então gerar os nossos chunks. Com os parâmetros que usamos, dados o(s) nosso(s) documento(s), estaremos dividindo ele em chunks de 1000 caracteres no máximo, com sobreposição de 200 caracteres. Vamos ver a mágica acontecendo.

In [28]:
chunks = text_splitter.split_documents(docs)

In [29]:
len(chunks), type(chunks)

(66, list)

Se você notar, a variável chunks é uma lista com 66 elementos. Esses elementos, conforme vemos abaixo, são instâncias da classe Document, assim como o seu documento carregado inicialmente. No entanto, agora temos mais porque os chunks são representações de menor tamanho do documento original, mas que o representam em sua totalidade.

**Adendo**:
Se tivéssemos uma string pura, ou um arquivo de texto lido diretamente, para gerarmos chunks dessa string pura precisamos utilizar um outro método:

In [33]:
string_pura = "ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fug ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fug"

chunks_2 = text_splitter.create_documents([string_pura]) # passamos dentro de uma lista

In [34]:
chunks_2

[Document(page_content='ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididu

### OpenAI Embeddings

Agora, a ideia é que geremos `embeddings`, isto é, as representações vetoriais semânticas, dos chunks que acabamos de criar. Aqui, estaremos utilizando o modelo de embeddings da OpenAI para gerá-los e faremos a conexão ainda por meio da classe que o LangChain disponibiliza para realizarmos essa requisição de forma fácil, o `OpenAIEmbeddings`.

Logo, temos:

#### 1. Instancie a classe com sua credencial (sua API Token) e sete o modelo que você quer utilizar.

In [37]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small", dimensions=1536)

Considerações:

1. Passamos a chave para conectar com a API. Lembre que a cada requisição para a API da OpenAI, haverá algum custo associado por requisição.
2. O modelo que utilizamos aqui foi o `text-embedding-3-small`, e é válido salientar que ele não é o único modelo de embedding da OpenAI. Em termos de desempenho e custo, ele está no intermedio, ou seja, não é o mais barato nem o mais caro, além de não ser o pior nem o melhor em questão de performance. Os outros que existem vou deixar como pesquisa, além de noções de custo.
3. As incorporações são feitas em um espaço n-dimensional. Por padrão, para esse modelo utilizado, as incorporações são feitas em um espaço dimensional de 1536 dimensões, mas se olhar a documentação da OpenAI, a incorporação pode ser feita para outro número de dimensões e a ideia é a mesma para os outros modelos de embeddings da OpenAI. Vale a pesquisa de quais são.|

#### 2. Gere os embeddings
Quando vamos gerar embeddings, geramos embeddings de textos (conteúdos textuais, strings). Passamos como parâmetro de fato uma lista, mas uma lista de strings. Não podemos passar a lista dos chunks diretamente porque os elementos da lista são do tipo Document, logo, basta fazermos:

In [39]:
embeddings_agent = embeddings.embed_documents([chunk.page_content for chunk in chunks])

In [42]:
embeddings_agent[0]

[0.009621397896576738,
 0.02036981535338998,
 0.04163011295203089,
 -0.004174142276702595,
 0.0034993226815892794,
 0.0019131485046835809,
 -0.014484273728097683,
 0.03673245128992475,
 -0.01904800221992821,
 0.058827577157641656,
 0.03720552219460549,
 -0.05256636607089427,
 -0.029274650844415696,
 -0.02938596252205597,
 -0.04374501247545355,
 -0.04911574131695001,
 -0.012918970025088234,
 -0.024112629535849542,
 0.04235363140611182,
 0.01622350006477485,
 0.04204752801789149,
 0.013426823966386349,
 0.03383838119889143,
 0.01844970940319258,
 -0.014414704860895118,
 -0.03125041077078802,
 0.00826480172649759,
 0.014059902613707166,
 -0.02331954128324344,
 -0.010122294858022338,
 0.05253853815148421,
 -0.018324483765847274,
 -0.031556514159008356,
 -0.018060122256742046,
 -0.0007017776592111679,
 0.03976566097800841,
 0.03753945350223589,
 0.003941086245611088,
 -0.025434440806666105,
 0.029274650844415696,
 0.003504540416478667,
 -0.06717585984840165,
 -0.02103767796865078,
 0.033281

In [45]:
len(embeddings_agent), len(embeddings_agent[0])
# quantos embeddings, quantas dimensões

(66, 1536)

E assim geramos embeddings dos nossos chunks/documentos.